# 关于Validator的使用教程

## 1.一些必要的准备工作
     我们首先从基于A股复权后日线的MACD模型开始。

In [1]:
# 准备好用于在回测过程中的k线读取工具，可以通过klinedata.read_data读取k线数据
# 这里值得注意的是：我们对日内周期的K线是的日期格式处理方式，
from QS.Calf import klinedata

default db: ppp have connected successfully


In [1]:
# 通过模型自有的IO工具读取用于验证的信号集
from QS.Macd import macddata as md
data = md.get_traitors(category='day_XDXR', dist=-70)

default db: ppp have connected successfully


In [3]:
# 其他一些自定义的内容，下以MACD模型为例
import datetime as dt
import pandas as pd
import numpy as np
data = data[((data.c_mark == False) & (data.rate < 1.03))]
data = data[(data.lcd2 < 0) & (data.lcd1 > 0)]
data = data[data.date > dt.datetime(2017, 1, 1)]
data['confidence'] = pd.eval('(data.last_max_area / data.max_area) * (data.last_max_macd / data.max_macd)')
data['confidence'] = np.log(data['confidence'])
data = data[data.confidence <= 7]

用于回测验证的信号片集已经准备就绪，它就是一个包含'code', 'type', 'open_price', 'open_date', 'confidence'字段的DataFrame

用于回测验证的信号片集已经准备就绪，它就是一个包含'code', 'type', 'open_price', 'open_date', 'confidence'字段的DataFrame

## 2.初始化验证器

In [4]:
# 引入验证器对象，由于MACD模型完全适用于modelvalidator类提供的原始回测方案
# 所以这里我们不需要继承modelvalidator再重写它的某些方法
from QS.Calf import modelvalidator as mv
sd = dt.datetime(2017, 1, 1)
ed = dt.datetime(2018, 1, 1)
# 初始化验证器，有关参数的说明，可以参考源代码
mv.VerifyFrame(data, 'XDXR_day', klinedata.read_data, sd, ed)

## 3.配置回测参数

In [5]:
# 关于各个参数的含义，可以参见modelvalidator的源代码
mv.modelparammodify(stop_get=0.07, stop_loss=0.05, max_pst_days=4, max_position=2)

## 4.执行验证

In [6]:
# 验证日内的信号时调用的方法是verify_min()
# 验证日线的信号时调用的方法是verify_day()
# 不考虑交易策略时调用的方法是verify_all()
# 使用上述方法其他步骤都是一样的
# 开始验证
dit, menu, goods = mv.verify_day()
# dit是一个关于本次回测的量化评估，menu是一个描述以日计的收益率的df
# goods是本次验证的交易清单

## 5.特殊环境下的回测验证
    这里所指的特殊环境是与中国A股交易规则不同的其他市场，比如外汇，美股等